In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context
from mand.core import ProfileMonitor, PrintMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import num, find
import datetime
from mand.lib.dbsetup import setUpDb

from mand.demos.trading import TradingBook, TradingPortfolio, MarketDataSource, MarketInterface

db = ObjectDb()
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x10b51aad0>: 105, mem=True, ro=False: entities=9, map=2


In [2]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['Eq-Prop', 'Eq-Inst', ])# 'FX', 'Rates', 'Credit', 'Delta1', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

# books: 20
# children: 2


In [3]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[0]
p2 = pAll.children()[1]

b1 = p1.children()[0]
b2 = p2.children()[0]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()

Customer1
Eq-Prop0
Eq-Inst0


In [4]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

In [5]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

In [6]:
with db:
    TradeOpenEvent = _tr.TradeOpenEvent
    cf1 = _tr.ForwardCashflow()
    ins1 = _tr.Equity()
    ins2 = _tr.Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

In [7]:
pm = None

class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            add('Starting Balance', valuable.NPV())
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: $%s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        displayHeader('%s' % title)
        displayListOfDicts(data, names=['Activity', 'PnL'] )
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: $1196.00

|Activity|PnL|
|-|-|
|Starting Balance|23.00
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

In [8]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

In [9]:
db3 = db.copy()
p = db3._get(pAll.meta.path())
    
# Note misleading cum times below due to recursion...

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times


# PnL explain for TopOfTheHouse: $2216.00

|Activity|PnL|
|-|-|
|Starting Balance|23.00
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:items|33|2,493,550|284|75,562|GetValue
|Portfolio:items|33|2,493,266|5,277|75,553|GetValue/Calc
|Report:data|1|1,473,689|13|1,473,689|GetValue
|Report:data|1|1,473,675|354|1,473,675|GetValue/Calc
|TradingContainer:NPV|11|1,468,446|118|133,495|GetValue
|TradingContainer:NPV|11|1,468,327|1,505|133,484|GetValue/Calc
|Workbook:items|224|1,010,766|1,940|4,512|GetValue
|Workbook:items|220|1,008,826|895,231|4,585|GetValue/Calc
|Root:Clocks|2|497,830|835|248,915|Context
|Portfolio:children|66|269,493|430|4,083|GetValue
|Portfolio:children|33|269,062|144,005|8,153|GetValue/Calc
|Equity:NPV|15|179,282|162|11,952|GetValue
|Equity:NPV|15|179,119|343|11,941|GetValue/Calc
|MarketInterface:spot|15|165,453|125|11,030|GetValue
|MarketInterface:spot|15|165,327|392|11,021|GetValue/Calc
|ExternalRefData:state|15|156,818|121|10,454|GetValue
|ExternalRefData:state|15|156,696|455|10,446|GetValue/Calc
|RefData:state|15|156,241|145|10,416|GetValue
|RefData:state|15|156,095|70,501|10,406|GetValue/Calc
|Clock:cutoffs|544|131,490|2,003|241|GetValue
|Clock:cutoffs|20|129,486|404|6,474|GetValue/Calc
|Root:End|1|127,113|37|127,113|Context
|Root:Start|1|120,468|34|120,468|Context
|Root:Activity Portfolio|1|107,919|41|107,919|Context
|Root:Activity Trading|1|106,992|36|106,992|Context
|Root:Amend Trading|1|104,916|34|104,916|Context
|Root:Amend Portfolio|1|101,994|32|101,994|Context
|Root:Start breaks|1|101,799|35|101,799|Context
|Root:Activity MarketData|1|100,353|45|100,353|Context
|Root:Amend MarketData|1|100,223|34|100,223|Context
|Clock:parent|20|97,020|147|4,851|GetValue
|Clock:parent|20|96,873|78,885|4,843|GetValue/Calc
|TradingBook|22|87,234|87,234|3,965|Db.Get
|PortfolioUpdateEvent:children|33|85,546|530|2,592|GetValue
|RefDataUpdateEvent|9|39,015|39,015|4,335|Db.Get
|Clock|5|20,947|20,947|4,189|Db.Get
|TradeOpenEvent|4|18,976|18,976|4,744|Db.Get
|PortfolioUpdateEvent|4|18,719|18,719|4,679|Db.Get
|TradeOpenEvent:ticket|66|17,799|355|269|GetValue
|TradingTicket|4|17,443|17,443|4,360|Db.Get
|Equity:refdata|15|13,322|144|888|GetValue
|Equity:refdata|15|13,177|432|878|GetValue/Calc
|TradingBook:clock|440|11,414|3,956|25|GetValue
|_WorkItemEvent:book2|66|9,660|441|146|GetValue
|ClockEvent|2|9,588|9,588|4,794|Db.Get
|MarketInterface|2|8,504|8,504|4,252|Db.Get
|MarketInterface:source|15|8,116|112|541|GetValue
|MarketInterface:source|15|8,003|362|533|GetValue/Calc
|ClockEvent:parent|8|7,651|72|956|GetValue
|TradingBook:clock|220|7,457|3,255|33|GetValue/Calc
|MarketDataSource|2|7,447|7,447|3,723|Db.Get
|_WorkItemEvent:item|33|7,336|214|222|GetValue
|Equity|2|7,122|7,122|3,561|Db.Get
|TradingPortfolio|2|6,999|6,999|3,499|Db.Get
|MarketDataSource:clock|30|6,171|235|205|GetValue
|MarketDataSource:clock|15|5,935|203|395|GetValue/Calc
|Portfolio:clock|66|4,512|596|68|GetValue
|MarketInterface|1|4,137|4,137|4,137|Db.Put
|Portfolio:clock|33|3,915|481|118|GetValue/Calc
|RootClock|1|3,695|3,695|3,695|Db.Get
|TradeOpenEvent:premium|33|3,635|192|110|GetValue
|ForwardCashflow|1|3,443|3,443|3,443|Db.Get
|Portfolio:books|55|1,666|399|30|GetValue
|Portfolio:books|33|1,266|1,077|38|GetValue/Calc
|Event:amends|142|857|857|6|GetValue
|Entity:clock|40|552|345|13|GetValue
|_WorkItemEvent:book1|66|300|300|4|GetValue
|TradeOpenEvent:quantity|66|286|286|4|GetValue
|RefDataUpdateEvent:data|58|269|269|4|GetValue
|ForwardCashflow:NPV|11|240|188|21|GetValue
|Entity:clock|20|206|206|10|GetValue/Calc
|MarketInterface:sourceName|15|192|128|12|GetValue
|RootClock:cutoffs|52|181|181|3|GetValue
|TradeOpenEvent:action|33|173|173|5|GetValue
|TradeOpenEvent:unitPrice|33|158|158|4|GetValue
|Equity:assetName|15|103|103|6|GetValue
|MarketInterface:sourceName|15|64|64|4|GetValue/Calc
|ForwardCashflow:NPV|11|51|51|4|GetValue/Calc
|Report:valuable|1|6|6|6|GetValue
|Report:ts2|1|4|4|4|GetValue
|Report:ts1|1|3|3|3|GetValue